# Low-frequency DAS Processing Using [DASDASE](https://github.com/DASDAE)

This Jupyter Notebook is created for low-frequency processing of a [spool](https://dascore.org/tutorial/concepts.html#:~:text=read%20the%20docs!-,Data%20structures,-DASCore%20has%20two) of distributed acoustic sensing (DAS) data. It uses [DASCore](https://dascore.org/) package and the ```lf_das.py``` script and it is inspired by Dr. Ge Jin's [low frequency processing](https://github.com/DASDAE/DASLowFreqProcessing). 


<svg width="100%" height="1">
  <line x1="0" y1="0" x2="100%" y2="0" style="stroke:rgb(0,0,0);stroke-width:2" />
</svg>


#### Notes: 
1. Before using this notebook, make sure you have included the ```lf_das.py``` script in the current directory with this notebook and successfully installed DASCore using ```pip``` or ```conda```:
    ```python
    pip install dascore
    ```
    or
    ```python
    conda install dascore -c conda-forge
    ```   
2. Please find all supported I/O [here](https://dascore.quarto.pub/dascore/).
 

Current DASCore version: 0.0.12

Date: 08/17/2023


Contact: [Ahmad Tourei](https://github.com/ahmadtourei/)

ahmadtourei@gmail.com

In [3]:
# import libraries
import dascore as dc
import matplotlib.pyplot as plt
import numpy as np
import scipy

from time import time
from lf_das import LFProc, get_edge_effect_time, get_patch_time, waterfall_plot


### Get a spool of data to work on

In [4]:
# define data path and output folder locations and then pass the data directory to the spool
data_path = '/mnt/h/data'
output_data_folder =  '/mnt/h/results'
output_figure_folder = '/mnt/h/figures'

# get the spool of data form the defined data path (will index patches for the first time)
sp = dc.spool(data_path)

# print the contents of first 5 patches
content_df = sp.get_contents()
content_df.head()

,network,dims,data_category,cable_id,time_min,file_version,tag,file_format,path,data_type,instrument_id,station,time_max,time_step
0,,"time,distance",,,2023-02-19 18:08:55.318832,2.1,,PRODML,BM73-22_UTC_20230219_180855.318.h5,strain_rate,,,2023-02-19 18:09:55.317832,0 days 00:00:00.001000
1,,"time,distance",,,2023-02-19 18:09:55.318833,2.1,,PRODML,BM73-22_UTC_20230219_180955.318.h5,strain_rate,,,2023-02-19 18:10:55.317833,0 days 00:00:00.001000


### Get some metadata and define a sub spool (if needed)

In [5]:
# get sampling rate, channel spacing, and gauge length from the first patch
patch_0 = sp[0]
gauge_length = patch_0.attrs['gauge_length']
print("Gauge length = ", gauge_length)
channel_spacing = patch_0.attrs['d_distance']
print("Channel spacing = ", channel_spacing)
sampling_interval = patch_0.attrs['d_time']
print("Sampling interval = ", sampling_interval)
sampling_rate = 1/(sampling_interval / np.timedelta64(1, 's'))
print("Sampling rate = ", sampling_rate)

# select a sub-spool if you'd like to
t_1 = '2023-08-01 10:00:00'
t_2 = '2023-08-02 10:00:00'
ch_start = 400
ch_end = 1400
d_1 = patch_0.coords['distance'][ch_start] # in meter
d_2 = patch_0.coords['distance'][ch_end] # in meter
# or:
# d_1 = -115 # in meter
# d_2 = 2000 # in meter
sub_sp = sp.select(distance=(d_1, d_2), time=(t_1, t_2)) 


/home/ahmadtourei/anaconda3/envs/dascore/lib/python3.11/site-packages/tables/attributeset.py:291: DataTypeWarning: Unsupported type for attribute 'TriggeredMeasurement' in node 'Acquisition'. Offending HDF5 class: 8
  value = self._g_getattr(self._v_node, name)


Gauge length =  10.0
Channel spacing =  2.0419039726257324
Sampling interval =  1000000 nanoseconds
Sampling rate =  1000.0


### Get low-pass filter parameters

In [6]:
# define memory size that you'd like to dedicate to low-frequency processing 
memory_size = 10000 # in MB
patch_length = get_patch_time(memory_size=memory_size, sampling_rate=sampling_rate, num_ch=ch_end-ch_start)
print('patch_length = ', patch_length, str(' sec.'))

# define the target sampling interval in seconds
d_t = 10.0 # so, cutoff_freq = Nyq_new = 1/(2*d_t) = 0.05 hz

# define the desired tolerance for getting the edge time (smaller tolerance results a longer eliminated edges in each patch and higher accuracy. 1e-3 is recommended.)
tolerance = 1e-3
edge_buffer = get_edge_effect_time(sampling_interval=1/sampling_rate, total_T=patch_length, tol=tolerance, freq=1/d_t)
print('edge_buffer = ', edge_buffer, str(' sec.'))

# pass the spool to the LFProc class
lfp = LFProc(sub_sp)
lfp.update_processing_parameter(output_sample_interval=d_t, process_patch_size=int(patch_length/d_t), edge_buff_size=int(np.ceil(edge_buffer/d_t)))

# set the output folder - Caution: If you set delete_existing=True, you will remove the output_data_folder directory. If you set delete_existing=False, you need to make sure the previous result for the same time range does not exist in the the output_data_folder directory.
lfp.set_output_folder(output_data_folder, delete_existing=False)


patch_length =  208.33333333333334  sec.


TypeError: Field elements must be 2- or 3-tuples, got '0'

### Do low-frequency processing and process the desired portion of the spool

In [ ]:
tic = time()
# do lowpass processing on (t_1,t_2) time range
t_1 = np.datetime64('2023-08-01T10:00:00')
t_2 = np.datetime64('2023-08-02T00:00:00')
lfp.process_time_range(t_1,t_2)
toc = time()
print(f'processing time (sec): {toc-tic}')


### Merge the results before visualization

In [ ]:
sp_result = dc.spool(output_data_folder).chunk(time=None) 


### Visualize the low-pass filtered results

#### a) Seismogram using Matplotlib 

In [ ]:
# define the channel of interest
channel = 1300
ch_inx = channel - ch_start
filtered_data = sp_result[0].data[:, ch_inx]
n_samples = filtered_data.shape[0]
num_sec = int(n_samples*d_t)
t_filt = np.linspace(0, num_sec, n_samples, endpoint=False)

# define the channel range whose mean value you want to subtract from the DAS trace
ch_start_demean = 400
ch_end_demean = 600
demeaned = (filtered_data - np.mean(sp_result[0].data[:,ch_start_demean-ch_start:ch_end_demean-ch_end], axis=1)) 

# plot the result
plt.figure(figsize=(12,8))
plt.plot(t_filt, demeaned, label='Low-freq. signal - channel: ' + str(channel))
plt.legend(loc='best')
plt.ylabel('Strain rate (1/sec)')
plt.xlabel('Time (sec)')
plt.title('Filtered signal')
plt.grid(True)

# save the figure into the output_figure_folder directory
file_name_lowfreq = '/dascore_lowpass_' + str(int(d_t*2)) + 'sec_filter_channel' + str(channel) + '.jpeg'
plt.savefig(output_figure_folder + file_name_lowfreq, dpi=600, format='jpeg')
plt.show()


Apply a median filter to remove noise and microseismic events


In [ ]:
# define window length (number of samples) for the median filter
window_size = 9
median_filtered_signal = scipy.ndimage.median_filter(demeaned, size=window_size)       

plt.figure(figsize=(12,8))
plt.plot(t_filt, median_filtered_signal, label='Low-freq. signal - channel: ' + str(channel))
plt.legend(loc='best')
plt.ylabel('Strain rate (1/sec)')
plt.xlabel('Time (sec)')
# plt.ylim(-6e-6, 6e-6)
plt.title('Filtered signal')
plt.grid(True)

# save the figure into the output_figure_folder directory
file_name_lowfreq = '/dascore_lowpass_' + str(int(d_t*2)) + 'sec_filtered_median_filtered_channel' + str(channel) + '.jpeg'
plt.savefig(output_figure_folder + file_name_lowfreq, dpi=600, format='jpeg')
plt.show()


#### b) Waterfall plot using Matplotlib 

In [ ]:
# get the filtered results from the result spool
filtered_data = sp_result[0].data

# define the channel range whose mean you want to subtract from the DAS trace
ch_start_demean = 400
ch_end_demean = 600
mean_array = np.mean(filtered_data[:,ch_start_demean-ch_start:ch_end_demean-ch_start], axis=1)  # Calculate the mean along axis 1
mean_array = mean_array.reshape(-1, 1)  # Reshape mean_array to match the shape of full_array

# demean the results
demeaned_data = (filtered_data - mean_array)

# define the channles and seconds you want to plot
min_sec = 0
max_sec = 13990
min_ch = 400
max_ch = 1355
fig_title = "low-freq. DAS"
waterfall_plot(demeaned_data.T, min_sec, max_sec, min_ch-ch_start, max_ch-ch_start, ch_start, channel_spacing, 1185, 1/d_t, fig_title, output_figure_folder, "low_freq_raster_plot")


##### Apply a median filter to remove noise and microseismic events


In [ ]:
# define window length (number of samples) for the median filter
window_size = 5 # means 5*d_t seconds
median_filtered_signal = scipy.ndimage.median_filter(demeaned_data, size=window_size)  

# define the channles and seconds you want to plot
min_sec = 0
max_sec = 13990
min_ch = 400
max_ch = 1355
fig_title = "low-freq. DAS - Median filtered"
waterfall_plot(median_filtered_signal.T, min_sec, max_sec, min_ch-ch_start, max_ch-ch_start,1185, 1/d_t, fig_title, output_figure_folder, "low_freq_raster_plot_median_filtered")


#### c) Waterfall plot using DASCore

In [ ]:
# select a sub spool for visualization
min_ch = 400
max_ch = 1355
d_1 = sp_result[0].coords['distance'][min_ch-ch_start] # in meter
d_2 = sp_result[0].coords['distance'][max_ch-ch_start] # in meter
sub_sp_result = sp_result.select(distance=(d_1, d_2))

# get the filtered data from the saved result
sub_sp_result_merged = sub_sp_result.chunk(time=None)
filtered_data = sub_sp_result_merged[0].data

# define the channel range whose mean you want to subtract from the DAS trace
ch_start_demean = 400
ch_end_demean = 600
mean_array = np.mean(filtered_data[:,ch_start_demean-ch_start:ch_end_demean-ch_start], axis=1)  # Calculate the mean along axis 1
mean_array = mean_array.reshape(-1, 1)  # Reshape mean_array to match the shape of full_array

# demean the results
demeaned_data = (filtered_data - mean_array)

# make a new patch containing the demeaned_data 
filtered_demeaned = sub_sp_result_merged[0].new(data=demeaned_data)

# plot
filtered_demeaned.viz.waterfall(scale=0.01) # scale acts as a clipper for data
